# Modelo

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import sklearn
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
warnings.filterwarnings('ignore')


In [ ]:
data = pd.read_csv("./Data.csv", encoding='latin-1')

In [ ]:
data.head(2)

In [ ]:
# Transformación de datos a tipo fecha
data['Fin_del_viaje']= pd.to_datetime(data['Fin_del_viaje'])
data['Inicio_del_viaje']=pd.to_datetime(data['Inicio_del_viaje'])

In [ ]:
# Ordenamiento de valores por id de usuario e inicio del viaje 
data = data.sort_values(by=['Usuario_Id', 'Inicio_del_viaje'])

In [ ]:
# Obtener un dataframe como resultado de la agrupación de las veces que ha realizado viajes
size = pd.DataFrame(data.groupby(data[['Usuario_Id']].columns.tolist(),
                                 as_index=False).size())
size = size.rename(columns ={'size': 'num_reincidencia'})
size = size.reset_index()

In [ ]:
# Primeros registros de los usuarios y sus viajes
size.head(5)

In [ ]:
# Unión del dataframe principal con la reincidencia por usuario
data = pd.merge(data, size[['Usuario_Id','num_reincidencia']], how ='left', on='Usuario_Id')

In [ ]:
# Obtención de las personas que hayan tenido más de un viaje
Days_2more = data.loc[data['num_reincidencia']>1]
Days_2more = Days_2more.sort_values(by=['Usuario_Id', 'num_reincidencia'])
Days_2more = Days_2more.reset_index(drop = True)

# Sacar la diferencia de viajes entre el mismo usuario
avg_dias = [Days_2more['Inicio_del_viaje'][i] - Days_2more['Inicio_del_viaje'][i-1]
if Days_2more['Usuario_Id'][i]==Days_2more['Usuario_Id'][i-1]
else np.nan for i in range(1,len(Days_2more))]

# Agregar 0 cuando se encuentra un valor nulo
avg_dias.insert(0,np.nan)

# Obtener el número de días en su formato correcto
Days_2more['Avg_dias'] = avg_dias
Days_2more['Avg_dias'] = Days_2more['Avg_dias'].dt.days

#Obtener la media de los dias por usuario 
Days_2more = Days_2more[['Usuario_Id', 'Avg_dias']].groupby('Usuario_Id').mean().round(2)
Days_2more = Days_2more.reset_index()


In [ ]:
# Obtener la fecha máxima
maxdate = data['Inicio_del_viaje'].max()
# Obtener los registros que solo hayan tenido un viaje
Days_1only = data.loc[data['num_reincidencia'] == 1]
# Obtener la diferencia entre la fecha máxima y el inicio del viaje en días
delta = (Days_1only['Inicio_del_viaje'] - maxdate).dt.days
# Settear a 0 en caso de obtención de valores negativos
Days_1only['Avg_dias'] =[0 if x < 0 else x for x in delta]
Days_1only = Days_1only[['Usuario_Id', 'Avg_dias']]
Days_1only = Days_1only.reset_index(drop= True)

In [ ]:
# Concatenación de los registros con al menos 1 o 2 más usos al día
AVG_dias_reinc = pd.concat([Days_2more,Days_1only], axis= 0)
AVG_dias_reinc = AVG_dias_reinc.reset_index(drop = True)

In [ ]:
# Ordenamiento de viajes por usuario y fecha de inicio de viaje
data = data.sort_values(by = ['Usuario_Id', 'Inicio_del_viaje'])

# Obtención de la fecha de último uso para obtener los días desde su último viaje
dias_ultimo_uso = maxdate - (data[['Usuario_Id','Inicio_del_viaje']].groupby('Usuario_Id').last())
dias_ultimo_uso['Inicio_del_viaje'] = dias_ultimo_uso['Inicio_del_viaje'].dt.days
dias_ultimo_uso.columns = ['recency']
dias_ultimo_uso = dias_ultimo_uso.reset_index()


In [ ]:
dias_ultimo_uso

In [ ]:
# Consulta por usuario particular
data.loc[data['Usuario_Id'] == 2004533]

In [ ]:
# Consulta de de los días que tiene sin usar el servicio
AVG_dias_reinc.loc[AVG_dias_reinc['Usuario_Id'] == 2004533]

In [ ]:
# Obtener la media de la distancia de los viajes
distancia = pd.pivot_table(data, values = 'distancia', index = ['Usuario_Id'], aggfunc ='mean').fillna(0)
distancia= distancia.reset_index()

In [ ]:
# Obtener la media del total del uso del servicio
tiempo = pd.pivot_table(data, values = 'total_min', index = ['Usuario_Id'], aggfunc ='mean').fillna(0)
tiempo =tiempo.reset_index()

In [ ]:
# Obtener el conteo de la reinicidencia por usuario
reincidencia = pd.pivot_table(data, values = 'num_reincidencia', index = ['Usuario_Id'], aggfunc ='count').fillna(0)
reincidencia = reincidencia.reset_index()

In [ ]:
# Obtención del mes
data['mes'] = data['Inicio_del_viaje'].dt.month

In [ ]:
# Obtener una tabla transformando el mes / 10, consultar Proyecto_MiBici
mes = pd.pivot_table(data, values = 'mes', index = ['Usuario_Id'], aggfunc= lambda x: len(x.unique())/10).fillna(0)
mes = mes.reset_index()

In [ ]:
mes

In [ ]:
# Creación del dataset master con registros únicos para la obtención de los clústers
Master = data['Usuario_Id']
# Eliminación de duplicados
Master = Master.drop_duplicates()
Master = Master.reset_index()

In [ ]:
# Unión de las variables significativas al dateset principal en función de usuario
Master = pd.merge(Master, dias_ultimo_uso, how='left',on='Usuario_Id')
Master = pd.merge(Master, AVG_dias_reinc, how='left',on='Usuario_Id')
Master = pd.merge(Master, distancia, how='left',on='Usuario_Id')
Master = pd.merge(Master, tiempo, how='left',on='Usuario_Id')
Master = pd.merge(Master, reincidencia, how='left',on='Usuario_Id')
Master = pd.merge(Master, mes, how='left',on='Usuario_Id')

In [ ]:
# Variables significativas
Master.columns

In [ ]:
# Formación del dataset final
Master = Master[['Usuario_Id', 'recency', 'Avg_dias', 'distancia', 'total_min', 'num_reincidencia', 'mes']]

In [ ]:
Master

In [ ]:
# Obtener las columnas de tipo numérico
# para graficarlas mediante box plots
num_columns = Master._get_numeric_data().columns
plt.figure(figsize=(10,20))
for i,col in enumerate(num_columns,1):
     plt.subplot(13,1,i)
     sns.boxplot(Master[col])
     plt.ylabel(col)
plt.show()

In [ ]:
#z = np.abs(stats.zscore(Master['recency']))
#outliers = pd.Series(np.where(z>3, True, False),index=Master['recency'].index)
#Master = Master.loc[outliers==False]
#(outliers==True).sum()
#(outliers==False).sum()


In [ ]:
#z = np.abs(stats.zscore(Master['recency']))
#outliers = pd.Series(np.where(z>3, True, False),index=Master['recency'].index)
#Master = Master.loc[outliers==False]
#(outliers==True).sum()
#(outliers==False).sum()

In [ ]:
# Obtención de la asimetría
Master.skew()

In [ ]:
# Obtención de la asimetría tras agregar las columnas transformadas
Master['recency_t']= Master['recency']**(1/3)
Master['Avg_dias_t'] = Master['Avg_dias']**(1/5)
Master['distancia_t'] = Master['distancia']**1
Master['total_min_t'] = Master['total_min']**(1/3)
Master['num_reincidencia_t'] = Master['num_reincidencia']**(1/3)
Master['mes_t']= Master['mes']**(1)

Master.skew()

In [ ]:
# Obtención de la media general
Master.mean(axis = 0)

In [ ]:
# Asignación de ids
IDs = Master['Usuario_Id']

In [ ]:
# Creación del dataset final, utilizando las variables transformadas
Master_t = Master.drop(columns= {'Usuario_Id','recency', 'Avg_dias','distancia', 'total_min', 'num_reincidencia', 'mes'})
Master_t.head()

In [ ]:
# Llenar los valores nulos con 0
Master_t = Master_t.fillna(0)
# Estandarización de los valores
Master_scaled = sklearn.preprocessing.StandardScaler().fit_transform(Master_t)
Master_scaled = pd.DataFrame(Master_scaled)
Master_scaled.columns = Master_t.columns
Master_scaled.head()

In [ ]:
# Medias del dataset escalado
Master_scaled.mean(axis=0)

In [ ]:
# Confirmación de las columnas principales
Master_t.columns

In [ ]:
# Obtención de los valores máximos y mínimos
recency_min = Master_t['recency_t'].min()
recency_max = Master_t['recency_t'].max()
Avg_dias_min =Master_t['Avg_dias_t'].min()
Avg_dias_max =Master_t['Avg_dias_t'].max()
distancia_min =Master_t['distancia_t'].min()
distancia_max =Master_t['distancia_t'].max()
total_min_min =Master_t['total_min_t'].min()
total_min_max =Master_t['total_min_t'].max()
num_reincidencia_min =Master_t['num_reincidencia_t'].min()
num_reincidencia_max =Master_t['num_reincidencia_t'].max()
mes_min =Master_t['mes_t'].min()
mes_max =Master_t['mes_t'].max()

In [ ]:
# Creación del clúster con los valores mínimos
MinFirstCluster = Master_t[['recency_t', 'Avg_dias_t', 'distancia_t', 'total_min_t',
       'num_reincidencia_t', 'mes_t']].min()
MinFirstCluster = pd.DataFrame(MinFirstCluster)
MinFirstCluster.columns = ["Minimums"]
MinFirstCluster = MinFirstCluster.reset_index()
MinFirstCluster

In [ ]:
# Creación del clúster con los valores máximos
MaxFirstCluster = Master_t[['recency_t', 'Avg_dias_t', 'distancia_t', 'total_min_t',
       'num_reincidencia_t', 'mes_t']].max()
MaxFirstCluster = pd.DataFrame(MaxFirstCluster)
MaxFirstCluster.columns = ["Maximums"]
MaxFirstCluster = MaxFirstCluster.reset_index()
MaxFirstCluster

In [ ]:
# Unión de los clústers mínimo y máximo
MinMaxFirstCLuster = pd.merge(MinFirstCluster, MaxFirstCluster, how='left', on = 'index')
MinMaxFirstCLuster

In [ ]:
# Definir la transformación a realizar/realizada en el dataset
MinMaxFirstCLuster['Cluster'] = 'Initial'
MinMaxFirstCLuster['Transformation'] = 0
MinMaxFirstCLuster['Transformation'][0] = 1/3
MinMaxFirstCLuster['Transformation'][1] = 1/5
MinMaxFirstCLuster['Transformation'][2] = 1
MinMaxFirstCLuster['Transformation'][3] = 1/3
MinMaxFirstCLuster['Transformation'][4] = 1/3
MinMaxFirstCLuster['Transformation'][5] = 1

MinMaxFirstCLuster

In [ ]:
# Verificar las columnas del dataset principal escalado
Master_scaled.columns

In [ ]:
# Aplicación del método del codo para la obtención de número de clúster óptimos
distortions = []
K = range(1,8)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(Master_scaled)
    distortions.append(kmeanModel.inertia_)
    
plt.figure(figsize = (16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.title('Grafica de codo mostrando la K optima')
plt.show()

In [ ]:
# Creación de la instancia del modelo Kmeans 
Master_scaled = Master_scaled
kmeans = KMeans()
# Generar los clusters dado el dataset escalado
Master_scaled["Cluster"] = kmeans.fit_predict(Master_scaled)
Master_scaled["Cluster"] = Master_scaled["Cluster"].astype("category")

In [ ]:
# Obtención de los centeroides
centroids = kmeans.cluster_centers_
centroids

In [ ]:
# Exportar el modelo 
pickle.dump(kmeans, open('clasificador.pkl', 'wb'))

In [ ]:
# Dataset principal una vez asignado el clúster
Master_scaled

In [ ]:
# Conteo de valores de mes_t en función de cluster
conteo = pd.pivot_table(Master_scaled, values = 'mes_t', index = ['Cluster'], aggfunc ='count')
conteo = conteo.reset_index()

In [ ]:
conteo

In [ ]:
# Unión del dataset principal con el cluster 
Master = pd.merge(Master, Master_scaled['Cluster'], right_index = True, left_index=True)

In [ ]:
# Confirmación de las columnas finales 
Master.columns

In [ ]:
# Descripción de los clusters a detalle 
Grupos = pd.pivot_table(Master, values = ['recency', 'Avg_dias', 'distancia', 'total_min', 'num_reincidencia', 'mes'],
                        index = ['Cluster'], aggfunc ='mean')
Grupos = Grupos.reset_index()
Grupos

In [ ]:
# Exportación de los datos con el formato necesario para el modelo
Master.to_csv('DataModelo.csv',encoding='UTF-8')
print("Success")